In [233]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [234]:
TICK_DATA = os.path.join(os.getcwd(), '2.csv')
ORDER_DATA = os.path.join(os.getcwd(), 'Assignment#3_Orders.csv')

tick = pd.read_csv(TICK_DATA, encoding='utf-16-le')
order = pd.read_csv(ORDER_DATA)

In [235]:
order.head()

,trade_id,buy/sell,notional,price,datetime,instrument,alpha
0,107414,1,"150,000",1.35724,1/7/2018 19:07:49.749,GBP/USD,SOM
1,107427,1,"310,000",1.35725,1/7/2018 19:15:23.1523,GBP/USD,SOM
2,107638,-1,"310,000",1.35453,1/8/2018 7:38:48.3848,GBP/USD,SOM
3,107649,-1,"310,000",1.35371,1/8/2018 8:10:55.1055,GBP/USD,SOM
4,107654,-1,"320,000",1.35394,1/8/2018 8:19:10.1910,GBP/USD,SOM


In [236]:
tick['DateTime'] = pd.to_datetime(tick['DateTime'], format="%m/%d/%Y %H:%M:%S.%f")
order['DateTime'] = pd.to_datetime(order['datetime'])

In [237]:
order = order.drop(['datetime'], axis=1).set_index('DateTime')
tick = tick.set_index('DateTime')

In [238]:
order.head()

,trade_id,buy/sell,notional,price,instrument,alpha
DateTime,,,,,,
2018-01-07 19:07:49.749000,107414,1,"150,000",1.35724,GBP/USD,SOM
2018-01-07 19:15:23.152300,107427,1,"310,000",1.35725,GBP/USD,SOM
2018-01-08 07:38:48.384800,107638,-1,"310,000",1.35453,GBP/USD,SOM
2018-01-08 08:10:55.105500,107649,-1,"310,000",1.35371,GBP/USD,SOM
2018-01-08 08:19:10.191000,107654,-1,"320,000",1.35394,GBP/USD,SOM


In [241]:
order = order.iloc[2:]

In [242]:
tick.head() 

,Bid,Ask
DateTime,,
2018-01-07 22:02:03.832,1.35560,1.35616
2018-01-07 22:02:03.845,1.35560,1.35617
2018-01-07 22:02:03.851,1.35554,1.35617
2018-01-07 22:02:03.859,1.35554,1.35618
2018-01-07 22:02:04.114,1.35554,1.35619


# Market Taking

In [197]:
def MT(order, tick):
    PnL_df = pd.DataFrame(index=order.index)
    
    for i in order.index:
        nearest_row_num = tick.index.get_loc(i, method='backfill') ## find nearest by DateTime
        nearest = tick.iloc[nearest_row_num] ## nearest fulfilled bid/ask price
        PnL = (nearest[0] - nearest[1])*1/2
        time = tick.index[nearest_row_num] ## time of fulfilled trade
        
        PnL_df.loc[i, 'MT PnL'] = PnL
        PnL_df.loc[i, 'MT Executed Time'] = time
        
    return PnL_df

In [198]:
MT_PnL = MT(order, tick)

In [199]:
MT_PnL.head()

,MT PnL,MT Executed Time
DateTime,,
2018-01-07 19:07:49.749000,-0.000280,2018-01-07 22:02:03.832
2018-01-07 19:15:23.152300,-0.000280,2018-01-07 22:02:03.832
2018-01-08 07:38:48.384800,-0.000010,2018-01-08 07:38:49.708
2018-01-08 08:10:55.105500,-0.000020,2018-01-08 08:10:55.233
2018-01-08 08:19:10.191000,-0.000015,2018-01-08 08:19:10.599


# Opportunistic Market Making

## OMMMid

In [229]:
tick.head()

,Bid,Ask
DateTime,,
2018-01-07 22:02:03.832,1.35560,1.35616
2018-01-07 22:02:03.845,1.35560,1.35617
2018-01-07 22:02:03.851,1.35554,1.35617
2018-01-07 22:02:03.859,1.35554,1.35618
2018-01-07 22:02:04.114,1.35554,1.35619


In [230]:
order = order.iloc[2:]

In [231]:
order.head()

,trade_id,buy/sell,notional,price,instrument,alpha
DateTime,,,,,,
2018-01-08 08:34:40.344000,107670,-1,"490,000",1.35423,GBP/USD,SOM
2018-01-08 11:25:13.251300,107726,1,"200,000",1.35716,GBP/USD,SOM
2018-01-08 11:42:08.428000,107733,1,"310,000",1.35693,GBP/USD,SOM
2018-01-08 12:15:34.153400,107737,1,"310,000",1.35676,GBP/USD,SOM
2018-01-08 12:15:35.153500,107738,1,"310,000",1.35677,GBP/USD,SOM


In [253]:
## Max Time to Execute
from datetime import timedelta
TTE = timedelta(seconds=10)
## Max Stop Loss
SL = 0.00003

In [254]:
SL = 0.00003

In [222]:
TTE

datetime.timedelta(0, 10)

In [146]:
SL

3e-05

In [147]:
order.index[0]

Timestamp('2018-01-08 07:38:48.384800')

In [148]:
order.index[0] + timedelta(seconds=10)

Timestamp('2018-01-08 07:38:58.384800')

In [149]:
order.index[0] - timedelta(seconds=10)

Timestamp('2018-01-08 07:38:38.384800')

In [55]:
i = order.index[100]
i

Timestamp('2018-01-10 05:38:25')

In [153]:
nearest_row_num = tick.index.get_loc(i, method='pad')
nearest_row_num

513118

In [154]:
nearest = tick.iloc[nearest_row_num]
nearest

Bid    1.35311
Ask    1.35314
Name: 2018-01-10 05:38:24.398000, dtype: float64

In [152]:
mid = (nearest[0] + nearest[1])/2
mid

1.3531300000000002

In [155]:
tick['buy loss'] = tick['Ask'] - mid
tick['sell loss'] = mid - tick['Bid'] 

In [156]:
tick.head()

,Bid,Ask,buy loss,sell loss
DateTime,,,,
2018-01-07 22:02:03.832,1.35560,1.35616,0.00303,-0.00247
2018-01-07 22:02:03.845,1.35560,1.35617,0.00304,-0.00247
2018-01-07 22:02:03.851,1.35554,1.35617,0.00304,-0.00241
2018-01-07 22:02:03.859,1.35554,1.35618,0.00305,-0.00241
2018-01-07 22:02:04.114,1.35554,1.35619,0.00306,-0.00241


In [158]:
order_time = i
order_time

Timestamp('2018-01-10 05:38:25')

In [159]:
end_time = order_time + TTE
end_time

Timestamp('2018-01-10 05:38:55')

In [160]:
TTE_time = tick.index[tick.index.get_loc(end_time, method='backfill')]
TTE_time

Timestamp('2018-01-10 05:38:55.045000')

In [164]:
near_order_time = (tick.iloc[tick.index.get_loc(order_time, method='backfill')]).name
near_order_time

Timestamp('2018-01-10 05:38:25.153000')

In [163]:
SL_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['buy loss']>SL].index[0]
SL_time

Timestamp('2018-01-10 05:38:33.080000')

In [165]:
mid_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['Ask']<=mid].index[0]
mid_time

Timestamp('2018-01-10 05:38:55.690000')

In [66]:
display(start_time, mid_time, SL_time, TTE_time)

Timestamp('2018-01-10 05:38:25.153000')

Timestamp('2018-01-10 05:38:55.690000')

Timestamp('2018-01-10 05:38:33.080000')

Timestamp('2018-01-10 05:38:35.126000')

In [166]:
trade_time = max(mid_time, SL_time, TTE_time)
trade_time

Timestamp('2018-01-10 05:38:55.690000')

In [167]:
PnL = mid - tick.loc[trade_time, 'Ask']
PnL

2.220446049250313e-16

In [128]:
(tick.iloc[tick.index.get_loc(start_time, method='backfill')]).name


Timestamp('2018-01-10 05:38:25.153000')

In [132]:
order.index[2:]

DatetimeIndex(['2018-01-08 07:38:48.384800', '2018-01-08 08:10:55.105500',
               '2018-01-08 08:19:10.191000', '2018-01-08 08:23:16.231600',
               '2018-01-08 08:34:40.344000', '2018-01-08 11:25:13.251300',
               '2018-01-08 11:42:08.428000', '2018-01-08 12:15:34.153400',
               '2018-01-08 12:15:35.153500', '2018-01-08 12:20:20.202000',
               ...
               '2018-01-12 13:04:03.430000', '2018-01-12 14:06:24.624000',
               '2018-01-12 14:33:14.331400', '2018-01-12 15:02:01.210000',
               '2018-01-12 15:17:02.172000', '2018-01-12 15:32:12.321200',
               '2018-01-12 16:03:23.323000', '2018-01-12 16:03:24.324000',
               '2018-01-12 16:18:07.187000', '2018-01-12 16:33:22.332200'],
              dtype='datetime64[ns]', name='DateTime', length=367, freq=None)

In [136]:
nearest_row_num = tick.index.get_loc(i, method='pad')
nearest_row_num

513118

In [246]:
def OMMMid(order, tick, SL, TTE):
    PnL_df = pd.DataFrame(index=order.index)
    
    max_time = tick.index[-1]+timedelta(days=1)
        
    for i in order.index:
        nearest_row_num = tick.index.get_loc(i, method='pad')
        nearest = tick.iloc[nearest_row_num]
        mid = (nearest[0] + nearest[1])/2
        
#         start_time = tick.index[nearest_row_num]
        order_time = i
        end_time = order_time + TTE
        TTE_time = tick.index[tick.index.get_loc(end_time, method='backfill')]
        
        near_order_time = (tick.iloc[tick.index.get_loc(order_time, method='backfill')]).name
        
        if order.loc[i,'buy/sell']==1:
            try: SL_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['buy loss']>=SL].index[0]
            except: SL_time = max_time
            try: mid_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['Ask']<=mid].index[0]
            except: mid_time = max_time
            trade_time = min(mid_time, SL_time, TTE_time)
        
            PnL = mid - tick.loc[trade_time, 'Ask']
        elif order.loc[i,'buy/sell']==-1:
            try: SL_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['sell loss']>=SL].index[0]
            except: SL_time = max_time
            try: mid_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['Bid']>=mid].index[0]
            except: mid_time = max_time
            trade_time = min(mid_time, SL_time, TTE_time)
        
            PnL = tick.loc[trade_time, 'Bid'] - mid
        
        if trade_time == TTE_time: trigger = 'TTE'
        elif trade_time == SL_time: trigger = 'SL'
        else: trigger = 'Else'
        
        PnL_df.loc[i, 'OMMMid PnL'] = PnL
        PnL_df.loc[i, 'OMMMid Time to Execute'] = trade_time - order_time
        PnL_df.loc[i, 'OMMMid Trigger'] = trigger
    
    return PnL_df

In [247]:
OMMMid_PnL = OMMMid(order, tick, SL, TTE)

In [252]:
OMMMid_PnL.head(10)

,OMMMid PnL,OMMMid Time to Execute,OMMMid Trigger
DateTime,,,
2018-01-08 07:38:48.384800,1.500000e-05,00:00:02.022200,Else
2018-01-08 08:10:55.105500,5.000000e-06,00:00:00.694500,Else
2018-01-08 08:19:10.191000,-9.000000e-05,00:00:10.214000,TTE
2018-01-08 08:23:16.231600,-1.500000e-05,00:00:13.521400,TTE
2018-01-08 08:34:40.344000,2.220446e-16,00:00:00.477000,Else
2018-01-08 11:25:13.251300,-5.500000e-05,00:00:10.142700,TTE
2018-01-08 11:42:08.428000,-7.000000e-05,00:00:10.446000,TTE
2018-01-08 12:15:34.153400,1.000000e-05,00:00:04.111600,Else
2018-01-08 12:15:35.153500,1.000000e-05,00:00:03.111500,Else


## OMMSide

In [255]:
def OMMSide(order, tick, SL, TTE):
    PnL_df = pd.DataFrame(index=order.index)
    
    max_time = tick.index[-1]+timedelta(days=1)
    
    for i in order.index:
        nearest_row_num = tick.index.get_loc(i, method='backfill')
        nearest = tick.iloc[nearest_row_num]
        mid = (nearest[0] + nearest[1])/2
        buy = nearest[0] ## Bid
        sell = nearest[1] ## Ask
        
        order_time = i
        end_time = order_time + TTE
        TTE_time = tick.index[tick.index.get_loc(end_time, method='backfill')]

        near_order_time = (tick.iloc[tick.index.get_loc(order_time, method='backfill')]).name
        
        if order.loc[i,'buy/sell']==1:
            try: SL_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['buy loss']>=SL].index[0]
            except: SL_time = max_time
            try: buy_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['Ask']<=buy].index[0]
            except: buy_time = max_time
            trade_time = min(buy_time, SL_time, TTE_time)
        
            PnL = mid - tick.loc[trade_time, 'Ask']
        elif order.loc[i,'buy/sell']==-1:
            try: SL_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['sell loss']>=SL].index[0]
            except: SL_time = max_time
            try: sell_time = (tick.loc[near_order_time:])[(tick.loc[near_order_time:])['Bid']>=sell].index[0]
            except: sell_time = max_time
            trade_time = min(sell_time, SL_time, TTE_time)
        
            PnL = tick.loc[trade_time, 'Bid'] - mid
        
        if trade_time == TTE_time: trigger = 'TTE'
        elif trade_time == SL_time: trigger = 'SL'
        else: trigger = 'Else'
        
        PnL_df.loc[i, 'OMMSide PnL'] = PnL
        PnL_df.loc[i, 'OMMSide Time to Execute'] = trade_time - order_time
        PnL_df.loc[i, 'OMMSide Trigger'] = trigger
    
    return PnL_df

In [256]:
OMMSide_PnL = OMMSide(order, tick, SL, TTE)

In [257]:
OMMSide_PnL.head()

,OMMSide PnL,OMMSide Time to Execute,OMMSide Trigger
DateTime,,,
2018-01-08 07:38:48.384800,0.000020,00:00:02.022200,Else
2018-01-08 08:10:55.105500,0.000020,00:00:05.976500,Else
2018-01-08 08:19:10.191000,-0.000095,00:00:10.214000,TTE
2018-01-08 08:23:16.231600,-0.000010,00:00:13.521400,TTE
2018-01-08 08:34:40.344000,0.000005,00:00:00.534000,Else


# Analysis

In [75]:
PnL = pd.concat([order['trade_id'], MT_PnL, OMMSide_PnL, OMMMid_PnL], axis=1)

In [76]:
PnL.head()

,trade_id,MT PnL,MT Executed Time,OMMSide PnL,OMMSide Time to Execute,OMMSide Trigger,OMMMid PnL,OMMMid Time to Execute,OMMMid Trigger
DateTime,,,,,,,,,
2018-01-07 19:07:49,107414,-0.000280,2018-01-07 22:02:03.832,-0.000280,00:00:00,SL,-0.000280,00:00:00,SL
2018-01-07 19:15:23,107427,-0.000280,2018-01-07 22:02:03.832,-0.000280,00:00:00,SL,-0.000280,00:00:00,SL
2018-01-08 07:38:48,107638,-0.000010,2018-01-08 07:38:49.708,0.000020,00:00:00.699000,Else,0.000000,00:00:00.079000,Else
2018-01-08 08:10:55,107649,-0.000020,2018-01-08 08:10:55.233,0.000020,00:00:05.849000,Else,0.000020,00:00:05.849000,Else
2018-01-08 08:19:10,107654,-0.000015,2018-01-08 08:19:10.033,-0.000105,00:00:09.437000,TTE,-0.000105,00:00:09.437000,TTE


In [77]:
PnL['minimum PnL'] = PnL.loc[:, ['MT PnL', 'OMMSide PnL', 'OMMMid PnL']].min(axis=1)
PnL['average PnL'] = PnL.loc[:, ['MT PnL', 'OMMSide PnL', 'OMMMid PnL']].mean(axis=1)
PnL['median PnL'] = PnL.loc[:, ['MT PnL', 'OMMSide PnL', 'OMMMid PnL']].median(axis=1)

In [78]:
PnL.head()

,trade_id,MT PnL,MT Executed Time,OMMSide PnL,OMMSide Time to Execute,OMMSide Trigger,OMMMid PnL,OMMMid Time to Execute,OMMMid Trigger,minimum PnL,average PnL,median PnL
DateTime,,,,,,,,,,,,
2018-01-07 19:07:49,107414,-0.000280,2018-01-07 22:02:03.832,-0.000280,00:00:00,SL,-0.000280,00:00:00,SL,-0.000280,-0.000280,-0.000280
2018-01-07 19:15:23,107427,-0.000280,2018-01-07 22:02:03.832,-0.000280,00:00:00,SL,-0.000280,00:00:00,SL,-0.000280,-0.000280,-0.000280
2018-01-08 07:38:48,107638,-0.000010,2018-01-08 07:38:49.708,0.000020,00:00:00.699000,Else,0.000000,00:00:00.079000,Else,-0.000010,0.000003,0.000000
2018-01-08 08:10:55,107649,-0.000020,2018-01-08 08:10:55.233,0.000020,00:00:05.849000,Else,0.000020,00:00:05.849000,Else,-0.000020,0.000007,0.000020
2018-01-08 08:19:10,107654,-0.000015,2018-01-08 08:19:10.033,-0.000105,00:00:09.437000,TTE,-0.000105,00:00:09.437000,TTE,-0.000105,-0.000075,-0.000105
